In [1]:
import pandas as pd
import numpy as np
import io
import mercury as mr #using mercurey lib
import matplotlib.pyplot as plt

movies = pd.read_csv(r"G:\Practice_code\nlp\tmdb_5000_movies.csv")
credits = pd.read_csv(r"G:\Practice_code\nlp\tmdb_5000_credits.csv")

In [2]:
movies = movies.merge(credits,on = 'title')
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.dropna(inplace = True)
movies.isnull().sum()
movies.duplicated().sum()

import ast

def convert(text):
    L=[]
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] =movies['keywords'].apply(convert)

def convert_2(text):
    L =[]
    counter = 0
    for i in ast.literal_eval(text):
        if counter<3:
            L.append(i['name'])
        counter +=1
    return L
movies['cast'] = movies['cast'].apply(convert_2)
#print(movies.head())

def convert_3(text):
    L=[]
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L
movies['crew'] = movies['crew'].apply(convert_3)
movies['overview'] = movies['overview'].apply(lambda x:x.split())

def remove_space(L):
    L1=[]
    for i in L:
        L1.append(i.replace(" ",""))
    return L1


In [3]:
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)

movies['Mark'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

new_movies = movies[['movie_id','title','Mark']]
new_movies['Mark'] = movies['Mark'].apply(lambda x: " ".join(x))
new_movies['Mark'] = new_movies['Mark'].apply(lambda x:x.lower())

C:\Users\PIUS\AppData\Local\Temp\ipykernel_21072\3513137135.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['Mark'] = movies['Mark'].apply(lambda x: " ".join(x))
C:\Users\PIUS\AppData\Local\Temp\ipykernel_21072\3513137135.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['Mark'] = new_movies['Mark'].apply(lambda x:x.lower())


In [7]:
query=input("Enter your query: ")
from sklearn.feature_extraction.text import CountVectorizer
cv =CountVectorizer(max_features=5000,stop_words='english')
vec = cv.fit_transform(new_movies['Mark']).toarray()
vec[1]
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stemming(text):
    list2 = []
    for i in text.split():
        list2.append(ps.stem(i))
    return " ".join(list2)
new_movies['Mark'] = new_movies['Mark'].apply(stemming)
new_movies['Mark']
vector1 = cv.fit_transform(new_movies['Mark']).toarray()
vector1[1]
#cv.get_feature_names()
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector1)
similarity

def recommended(movie):
    fetch_index = new_movies[new_movies['title'] == movie].index[0]
    distance = similarity[fetch_index]
    final_list = sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    for i in final_list:
        print(new_movies.iloc[i[0]].title)
recommended(query)

Enter your query: Batman Begins


C:\Users\PIUS\AppData\Local\Temp\ipykernel_21072\2072656329.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['Mark'] = new_movies['Mark'].apply(stemming)


The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf
